In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# Load the dataset
tweets_df = pd.read_csv('data/data_for_spcuban_discriminator.csv')
tweets_df.head()

In [ ]:
tweets = tweets_df['tweet']
labels = tweets_df['label']
tweets_train, tweets_test, labels_train, labels_test = train_test_split(tweets, labels, test_size=0.1, random_state=42)
tweets_train, tweets_val, labels_train, labels_val = train_test_split(tweets_train, labels_train, test_size=0.1, random_state=42)

In [ ]:
# create dataset instances
train_dataset = Dataset.from_pandas(pd.DataFrame({'tweet': tweets_train, 'label': labels_train}))
val_dataset = Dataset.from_pandas(pd.DataFrame({'tweet': tweets_val, 'label': labels_val}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'tweet': tweets_test, 'label': labels_test}))

In [ ]:
# Load the tokenizer and model (BETO)
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=2)

In [ ]:
# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',
    greater_is_better=True,
)

In [ ]:
# Define the trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,    
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
# Save the model
model.save_pretrained('models/spcuban_discriminator.pt')

In [1]:
print('Model saved!')

Model saved!
